In [75]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
import re

In [76]:
name = []
price = []
province = []
sold = []

In [77]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
driver = webdriver.Chrome(executable_path=r'C:\Users\Asus\Documents\DSI311\chromedriver.exe', options=options)


for i in range(0,10):
    driver.get(f'https://shopee.co.th/search?keyword=herb&page={i}')
    time.sleep(3)
    if i == 0:
        thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
        thai_button.click()
        time.sleep(2)
        
    driver.execute_script("document.body.style.zoom='25%'")
    time.sleep(3)
    data = driver.page_source
    soup = bs(data)
    
    all_item = soup.find_all("div",{'class':'col-xs-2-4 shopee-search-item-result__item'})
    
    for i in all_item:
        a = bs(str(i))
        
        #ชื่อ
        n = a.find('div',{'class':'ie3A+n bM+7UW Cve6sh'}).get_text() #ดึงชื่อจาก tag
        name.append(n)
        
        #ราคา
        p = a.find('span',{'class':'ZEgDH9'}).get_text() #ดึงราคา
        price.append(p)
        
        #จังหวัด
        pro = a.find('div',{'class':'zGGwiV'}).get_text()
        pro = re.sub("จังหวัด","",pro)
        province.append(pro)
        
        #ขายแล้ว
        s = a.find('div',{'class':'ZnrnMl'}).get_text()
        if s == "":
            s = "-1"
        sold.append(s)

AttributeError: 'NoneType' object has no attribute 'get_text'

In [78]:
#ปรับจำนวนการขายเป็นตัวเลข
amountsold = []
for i in sold:
    i = i.split()
    if len(i) == 1:
        amountsold.append(i[0])
    else:
        j = re.sub("พัน","k",i[1])
        if "k" in j:
            j = re.sub("k","",j)
            j = float(j)
            j *= 1000
            amountsold.append(j)
        else:
            j = float(j)
            amountsold.append(j)

#ปรับราคาเป็น float
price_num = []
for i in price:
    i = re.sub(",","",i)
    i = float(i)
    price_num.append(i)

#ปรับชื่อ
name_clean = []
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
for i in name:
    i = emoji_pattern.sub(r'', i)
    name_clean.append(i)

In [86]:
final = pd.DataFrame({'Name': name_clean,
                     'Price' : price_num,
                     'Province': province,
                     'AmountSold': amountsold})
final.set_index('Name')

,Price,Province,AmountSold
Name,,,
ฮัมเฮิร์บ พาวเวอร์ฟูล แอมพลู เซรั่ม AM HERB POWERFUL AMPOULE SERUM 20 ML.,339.0,บุรีรัมย์,6.0
♦️แถมฟรีดีท็อกซ์♦️ของแท้ส่งไว♦️เฮิร์บ​ HERB VIP Korse คอร์เซ่ คอเซ่ เฮิร์บ วี ไอ พี (กล่องซีล & ล็อตใหม่),88.0,กรุงเทพมหานคร,172500.0
G-HERB จี-เฮิร์บ จีเฮิร์บ แคปซูลวัน สมุนไพรบำรุงน้ำเหลือง สร้างภูมิคุ้มกัน สมุนไพรหมอสมหมาย 60 แคปซูล 1 กระปุก,900.0,กรุงเทพมหานคร,25.0
[3 ขวด ราคาพิเศษ] THAI HERB เคอร์คูมิน โปร ผลิตภัณฑ์เสริมอาหารผงขมิ้นชันผสมโปรไบโอติก (30 แคปซูล),900.0,พระนครศรีอยุธยา,35.0
DayHerb แชมพูปิดผมขาว สมุนไพรแท้ ปิดหงอก ลดหงอก ผมดำใน 5 นาที,790.0,นนทบุรี,5600.0
Me Herb Phlu Khao มีเฮิร์บ พลูคาว (30 แคปซูล/กระปุก) ผลิตภัณฑ์เสริมอาหาร เสริมภูมิคุ้มกัน สมุนไพรพลูคาว น้ำมันสกัดเย็น,350.0,ปทุมธานี,52.0
G-HERB จีเฮิร์บ (โฉมใหม่) สมุนไพรบำรุงน้ำเหลือง บำรุงร่างกาย เสริมภูมิคุ้มกัน คุณหมอสมหมาย,900.0,กรุงเทพมหานคร,34.0
แชมพูปิดผมขาว DAYHERB แชมพูปิดผมขาวสมุนไพร,465.0,กรุงเทพมหานคร,1400.0
G-HERB จี-เฮิร์บ จีเฮิร์บ แคปซูลวัน สมุนไพรบำรุงน้ำเหลือง สร้างภูมิคุ้มกัน สมุนไพรหมอสมหมาย 60 แคปซูล 1 กระปุก,900.0,กรุงเทพมหานคร,25.0


In [87]:
final

,Name,Price,Province,AmountSold
0,ฮัมเฮิร์บ พาวเวอร์ฟูล แอมพลู เซรั่ม AM HERB PO...,339.0,บุรีรัมย์,6.0
1,♦️แถมฟรีดีท็อกซ์♦️ของแท้ส่งไว♦️เฮิร์บ​ HERB VI...,88.0,กรุงเทพมหานคร,172500.0
2,G-HERB จี-เฮิร์บ จีเฮิร์บ แคปซูลวัน สมุนไพรบำร...,900.0,กรุงเทพมหานคร,25.0
3,[3 ขวด ราคาพิเศษ] THAI HERB เคอร์คูมิน โปร ผลิ...,900.0,พระนครศรีอยุธยา,35.0
4,DayHerb แชมพูปิดผมขาว สมุนไพรแท้ ปิดหงอก ลดหง...,790.0,นนทบุรี,5600.0
5,Me Herb Phlu Khao มีเฮิร์บ พลูคาว (30 แคปซูล/ก...,350.0,ปทุมธานี,52.0
6,G-HERB จีเฮิร์บ (โฉมใหม่) สมุนไพรบำรุงน้ำเหลือ...,900.0,กรุงเทพมหานคร,34.0
7,แชมพูปิดผมขาว DAYHERB แชมพูปิดผมขาวสมุนไพร,465.0,กรุงเทพมหานคร,1400.0
8,G-HERB จี-เฮิร์บ จีเฮิร์บ แคปซูลวัน สมุนไพรบำร...,900.0,กรุงเทพมหานคร,25.0
9,น้ำเหลืองไม่ดี ภูมิแพ้ สะเก็ดเงิน แผลหายช้า จ...,900.0,ฉะเชิงเทรา,103.0


In [88]:
final.to_csv('shopeescrap.csv',index = False)

NameError: name 'table' is not defined